In [165]:
from dotenv import load_dotenv
from agents import Agent, Runner,trace,function_tool , AsyncOpenAI, OpenAIChatCompletionsModel, set_tracing_disabled
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import os
import asyncio
import mimetypes
from email.message import EmailMessage

In [166]:
load_dotenv(override=True)

True

In [167]:
client = AsyncOpenAI(
    api_key=os.getenv("GEMINI_ACCESS_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)
model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=client
)

In [168]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [169]:
sales_agent_1=Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model=model
)

sales_agent_2=Agent(
    name="Engaging Sales Agent",
    instructions=instructions2,
    model=model
)

sales_agent_3=Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model=model
)

In [170]:
results=Runner.run(sales_agent_1, "Write a cold sales email")
results = await results
print(results.final_output)


# stream = await client.chat.completions.create(
#     model="gemini-2.0-flash",
#     messages=[
#         {"role": "system", "content": instructions2},  # from sales_agent_1
#         {"role": "user", "content": "Write a cold sales email"}
#     ],
#     stream=True
# )

# async for chunk in stream:
#     if chunk.choices and chunk.choices[0].delta.content:
#         print(chunk.choices[0].delta.content)

Subject: Streamlining SOC 2 Compliance & Audit Readiness with ComplAI

Dear [Prospect Name],

My name is [Your Name] and I'm a Sales Agent at ComplAI. We help companies like [Prospect Company's Industry/Type] achieve and maintain SOC 2 compliance faster, more efficiently, and with significantly less overhead through our AI-powered SaaS platform.

I understand that maintaining compliance with SOC 2, especially in anticipation of an audit, can be a complex and resource-intensive undertaking. Many companies struggle with:

*   **Documenting and managing controls:** Maintaining up-to-date documentation across all control areas.
*   **Evidence collection:** Gathering the necessary evidence to demonstrate ongoing compliance.
*   **Remediation efforts:** Identifying and addressing gaps in compliance efficiently.
*   **Audit preparation:** Consolidating information and preparing for the audit process.

ComplAI addresses these challenges head-on by automating evidence collection, providing real

In [138]:
message="Write a cold sales email"

with trace("Parallel code emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent_1,message),
        Runner.run(sales_agent_2,message),
        Runner.run(sales_agent_3,message),
    )

outputs= [result.final_output for result in results]
for output in outputs:
    print(output + "\n\n")

Subject: Streamline SOC 2 Compliance & Audit Preparation with ComplAI

Dear [Recipient Name],

In today's data-driven landscape, achieving and maintaining SOC 2 compliance is not just a best practice, but a critical requirement for establishing trust with your customers and partners. As [Recipient Title] at [Company Name], I understand that navigating the complexities of SOC 2 can be time-consuming and resource-intensive.

That's why I'm reaching out from ComplAI. We offer an AI-powered SaaS solution specifically designed to streamline your SOC 2 compliance journey and simplify audit preparation.

ComplAI helps you:

*   **Automate Evidence Collection:** Reduce manual effort by automatically collecting and organizing evidence required for SOC 2 controls.
*   **Identify & Remediate Gaps:** Proactively identify gaps in your security posture and receive actionable recommendations for remediation.
*   **Simplify Audit Preparation:** Generate audit-ready reports and documentation, significa

In [139]:
sales_picker=Agent(
    name="sales_picker",
    instructions="You pick the best cold email from the given oprions.\
    Imagine you are a customer and pick the one you are most likely to rspond to. \
    Do not give an explaination; reply with the selected email only.",
    model=model
)

In [140]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent_1,message),
        Runner.run(sales_agent_2,message),
        Runner.run(sales_agent_3,message),
    )
    output = [result.final_output for result in results]
    emails = "Cold sales emails:\n\n".join(outputs)

    best = await Runner.run(sales_picker,emails)

    print(f"Best sales email:\n{best.final_output}")

Best sales email:
Subject: SOC2 Compliance: Less Stress, More...Success? (ComplAI Can Help!)

Hey [Prospect Name],

Let's be honest, "SOC2 compliance" probably ranks somewhere between "root canal" and "untangling Christmas lights" on your list of favorite things to do. Am I right?

But fear not, brave leader of [Prospect Company]! We at ComplAI understand the struggle. We’re like the Gandalf to your Frodo, guiding you (minus the eagles, sorry) through the fiery depths of audits and security questionnaires.

ComplAI is an AI-powered SaaS platform that takes the "ugh" out of SOC2. We automate evidence collection, simplify policy creation, and generally make the whole process less painful (and more, dare I say, *enjoyable*).

**Here's the lowdown (in a nutshell):**

*   **AI Automation:** Let our AI do the heavy lifting. Think less manual spreadsheet madness, more sipping on a well-deserved latte.
*   **Continuous Monitoring:** We keep an eye on things so you don't have to. Consider us yo

<h2>Let's use Tools</h2>

In [171]:
sales_agent_1=Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model=model
)

sales_agent_2=Agent(
    name="Engaging Sales Agent",
    instructions=instructions2,
    model=model
)

sales_agent_3=Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model=model
)

<h2>Step 2: Tools and Agent interactions</h2>

In [172]:
from email.message import EmailMessage
import ssl
import smtplib

@function_tool
def send_email(body:str):
    msg = EmailMessage()
    msg.set_content(body)
    msg["Subject"] = "Sales Email"
    msg["From"] = "himanshusinghvns343@gmail.com"
    msg["To"] = "himanshusinghvns343@gmail.com"

    context = ssl.create_default_context()
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls(context=context)
        server.login("himanshusinghvns343@gmail.com", "sbcbpcezhmmornnn")
        server.send_message(msg)

    return {"status": "success"}

In [173]:
send_email

FunctionTool(name='send_email', description='', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1279bf100>, strict_json_schema=True, is_enabled=True)

<h4>We can convert an Agent into a tool</h4>

In [174]:
tool1=sales_agent_1.as_tool(tool_name="sales_agent_1",tool_description="Write a cold sales email")
tool1

FunctionTool(name='sales_agent_1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent_1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x1279bfce0>, strict_json_schema=True, is_enabled=True)

In [175]:
description="Write a cold sales email"

tool1=sales_agent_1.as_tool(tool_name="sales_agent_1",tool_description=description)
tool2=sales_agent_2.as_tool(tool_name="sales_agent_2",tool_description=description)
tool3=sales_agent_3.as_tool(tool_name="sales_agent_3",tool_description=description)

tools=[tool1,tool2,tool3,send_email]

tools

[FunctionTool(name='sales_agent_1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent_1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x127a1c0e0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent_2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent_2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x127a1c4a0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='sales_agent_3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'requ

In [176]:
instructions="You are a sales manager working for ComplAI. You use the tools given to you to generate cold sales emails.\
    You never generate sales emails yourself; you always use the tools.\
    You try all 3 sales_agent tools once before choosing the best one.\
    You pick the single best email and use the send_email tool to send the best email (and only the best email) to the user."

sales_manager= Agent(name="Sales Manager",instructions=instructions,tools=tools,model=model)

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager,message)

<h1>Handoffs</h1>
<h3>Handoffs represent a way an agent can delegate to an agent, passing control to it</h3>

Handoffs and Agents-as-tools are similar:<br><br>
In both cases, an Agent can colaborate with another Agent<br><br>
With tools, control passes back<br><br>
With handoffs, control passes across

In [177]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model=model)
subject_tool = subject_writer.as_tool(tool_name="subject_writer",tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body coverter",instructions=html_instructions,model=model)
html_tool=html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [178]:
@function_tool
def send_html_email(subject:str, html_body:str) ->Dict[str,str]:
    msg = EmailMessage()
    msg["Subject"] = subject
    msg["From"] = "himanshusinghvns343@gmail.com"
    msg["To"] = "himanshusinghvns343@gmail.com"
    msg.add_alternative(html_body, subtype="html")

    context = ssl.create_default_context()
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls(context=context)
        server.login(os.getenv('GMAIL_USER_ID'), os.getenv('GMAIL_API_KEY'))
        server.send_message(msg)

    return {"status": "success"}
    

In [179]:
tools=[subject_tool,html_tool,send_html_email]

In [180]:
instructions="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body. DO NOT include subject in body."

emailer_agent= Agent(
    name="Email Manager",
    instructions=instructions,
    tools=tools,
    model=model,
    handoff_description="Convert an email to HTML and send it"
)

In [181]:
tools=[tool1,tool2,tool3]
handoffs=[emailer_agent]

print(tools)

print(handoffs)

[FunctionTool(name='sales_agent_1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent_1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x127a1c0e0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent_2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent_2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x127a1c4a0>, strict_json_schema=True, is_enabled=True), FunctionTool(name='sales_agent_3', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'requir

In [182]:
sales_manager_instructions="""
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""

sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model=model
)

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager,message)